# Resources for acquiring and working with GEDI data
This brief tutorial summarizes the current learning resources available for acquiring and processing GEDI data in both R and Python. The GEDI science team and LPDAAC have created an R package and python scripts to aid working with GEDI data. Rather than reinvent the wheel this tutorial points to those existing resources and primarily demonstrates a workflow for using the two main LPDAAC tools for finding, downloading, and converting GEDI hdf files into a GIS format. 

## rGEDI
The rGEDI package (https://cran.r-project.org/web/packages/rGEDI/index.html) put together by Carlos Silva and members of the GEDI team like Steve Hancock contains everything necessary to find GEDI granules intersecting an area and time of interest, download the granules, and perform some basic analysis. A great tutorial on how to use rGEDI can already be found on their website: https://cran.r-project.org/web/packages/rGEDI/vignettes/tutorial.html

## pyGEDI
The pyGEDI package (https://github.com/EduinHSERNA/pyGEDI) includes several tools for downloading and converting GEDI HDF files and visualizing waveforms.

## LPDAAC Python tools and tutorials
The NASA LPDAAC which hosts the GEDI data has released a tool for finding GEDI granules that intersect an area of interest (GEDI Finder) and a python script to then subsetting the downloaded granule and converting it from HDF5 to a GIS data format (GEDI Subsetter).   

The **GEDI Finder** service allows you submit an HTTP query to find GEDI granules intersecting a given bounding box and returns a list of the granule download URLs. Details on using this service can be found on the GEDI finder website: https://lpdaacsvc.cr.usgs.gov/services/gedifinder and in the user guide: https://lpdaac.usgs.gov/documents/591/GEDIFinder_UserGuide_v1.0.pdf

The **GEDI Subsetter** tool is a python script that converts already downloaded GEDI granules in HDF5 format and converts them to GeoJSON files. This tool can be downloaded from the GEDI Subsetter tool website (https://git.earthdata.nasa.gov/projects/LPDUR/repos/gedi-subsetter/browse) where you'll also find instructions on how to use it. 

In addition to these tools the LPDAAC has released a series of tutorials (https://lpdaac.usgs.gov/news/release-getting-started-gedi-l1b-l2a-and-l2b-data-python-tutorial-series/) for GEDI L1B, L2A, and L2B, products that demonstrates downloading a single GEDI granule, conversion of the granule in a GeoDataframe, and basic analysis.

## GEDI Finder and Subsetter workflow  
This tutorial demonstrates a workflow for using the two primary LPDAAC tools for finding GEDI granules intersecting an area and time period of interest, downloading those granules, and then subsetting them to the area of interest and converting them to a GIS format. Once setup is finished, the remaining sections could be strung together into a reusable python script to streamline the process for different aoi's and time periods. 

### Setup Python Environment
Before working with these tools it's first necessary to create a python environment which contains all the necessary packages for submitting HTTP requests and running the GEDI Subsetter tool. Below is a modified copy of the environment setup instructions provided for the use of the Subsetter tool; the original instructions can be found here (https://git.earthdata.nasa.gov/projects/LPDUR/repos/gedi-subsetter/browse).

> #### 1. It is recommended to use [Conda](https://conda.io/docs/), an environment manager, to set up a compatible Python environment. Download Mini-Conda for your OS here: https://docs.conda.io/en/latest/miniconda.html. Once you have Conda installed, Follow the instructions below to successfully setup a Python environment on Windows, MacOS, or Linux.  
> #### 2. Download the GEDI_Subsetter.py script: https://git.earthdata.nasa.gov/projects/LPDUR/repos/gedi-subsetter/browse/GEDI_Subsetter.py
> #### 3. Download and extract the [Aria2 batch download tool](https://aria2.github.io/) - Zip files for pre-compiled builds of release 1.35.0 can be found under assets at the bottom of the page here: https://github.com/aria2/aria2/releases/tag/release-1.35.0  
> #### 4. Setup
> 1.  Open a new command line interface (MacOS/Linux: Terminal, Windows: Anaconda Command Prompt) and type: `conda create -n gedi -c conda-forge --yes python=3.7 h5py geopandas requests jupyterlab joblib matplotlib descartes`    
> TIP: Getting an error from the command line saying 'Conda is not an executable command' or something of that nature? Try re-opening a new terminal/Command Prompt.
> 2. Navigate to the directory where you downloaded the `GEDI_Subsetter.py` script
> 3. Activate GEDI Python environment (created in step 1) in the Command Prompt/Terminal  
  > Type:  Windows: `activate gedi` or MacOS: `source activate gedi`    
> TIP: Having trouble activating your environment, or loading specific packages once you have activated your environment? Try the following:
  > Type: 'conda update conda'    
  > [Additional information](https://conda.io/docs/user-guide/tasks/manage-environments.html) on setting up and managing Conda environments.  
> 4. Run `jupyter lab` in the command line to open a jupyterlab instance in the browser  
> 5. Either open this notebook in the same diretory or create a new jupyter notebook with `File > New > Notebook` and follow the code and instructions below

### Finding Granules
This demonstrates how to use the *requests* package to submit an HTTP query to the GEDI Finder tool, and then further subset the results to a date range. Details on how to use the GEDI finder tool can be found in the previously linked user guide or website.

In [ ]:
# Setup
import requests, os
basedir = r'G:\temp\gedi2' # Change to your base download directory
os.makedirs(basedir, exist_ok=True)

In [ ]:
# Create an aoi file to work with for this example
from shapely import wkt
import geopandas as gpd
aoi_path = os.path.join(basedir, 'aoi.shp')
aoi_dict = [{'id':1, 'geometry':wkt.loads('POLYGON ((-769057.0267802363 1978704.451861572, -766419.6818279801 1982728.676486997, -766135.4851736421 1980250.48166117, -763157.1042361801 1979761.663415708, -765567.091864966 1975350.931340384, -769057.0267802363 1978704.451861572))')},
            {'id':2, 'geometry':wkt.loads('POLYGON ((-760587.9664809648 1975009.895355177, -762781.964652454 1981182.646687398, -759996.8374399418 1982785.515817865, -757314.0210229913 1977022.00766789, -760587.9664809648 1975009.895355177), (-760974.4739308645 1980341.424590558, -760758.4844735676 1977692.711772128, -759280.6618710101 1978806.762657133, -759655.801454736 1980341.424590557, -760974.4739308645 1980341.424590558))')}]

aoi = gpd.GeoDataFrame(aoi_dict, crs=5070, geometry='geometry')

aoi.to_file(aoi_path)
aoi.plot()

In [ ]:
# Read the example aoi as you normally would
aoi = gpd.read_file(aoi_path)

# Reproject to WGS84 and dissolve into one feature
aoi_wgs84 = aoi.to_crs(epsg=4326)
bounds = aoi_wgs84.unary_union.bounds

Note that by transforming the aoi to WGS84 without buffering or densifying the polygons, some granules may be incorrectly included or excluded. This especially true for AOI's over large areas and with few vertices. To err on the side of caution and ensure that every granule intersecting the non-WGS84 AOI is included, buffer and densify the polygons before reprojecting to WGS84 and obtaining the bounding box.

In [ ]:
# Get URLs covering intersecting the bounding box of an AOI
bbox = [bounds[3], bounds[0], bounds[1], bounds[2]]  #  reorder to [ul_lat,ul_lon,lr_lat,lr_lon] in DD WGS84
bbox_str = str(bbox).replace(' ', '')
query = "https://lpdaacsvc.cr.usgs.gov/services/gedifinder?product=GEDI02_B&version=001&bbox="+bbox_str+"&output=json"
r = requests.get(query)
urls = r.json()['data']
urls

In [ ]:
# Filter the URLs based on time of year
startdoy = 152 # June 1st
enddoy = 258   # September 15th
date_urls = [url for url in urls if startdoy <= int(url.split('/')[-1][13:16]) <= enddoy]
date_urls

In [ ]:
# Save the list to a text file
url_path = os.path.join(basedir, "urls.txt")
with open(url_path, 'w') as f:
    f.write('\n'.join(date_urls))

### Downloading Granules  
The text file of URLs can be imported by batch download managers for executing the download. Here I show how to initiate the download from within python using [Aria2](https://aria2.github.io/). The LPDAAC also has a [script here](https://git.earthdata.nasa.gov/projects/LPDUR/repos/daac_data_download_python/browse) that helps with authentication and download, but it downloads files one at a time. This can be a time consuming process with many large files. Aria2 allows for concurrent downloads which should be much faster.

In [ ]:
# Download setup
from subprocess import Popen, CREATE_NEW_CONSOLE
aria_path = r"C:\Users\stevenf\Downloads\aria2-1.35.0-win-64bit-build1\aria2c.exe" # replace with your path to aria2c.exe
download_dir = os.path.join(basedir, "downloads")
os.makedirs(download_dir, exist_ok=True)

# Run this cell and provide your NASA Earth Data login at the prompt
# Authentication can also be done with a netrc file as described here: https://git.earthdata.nasa.gov/projects/LPDUR/repos/daac_data_download_python/browse
import getpass
nasa_username = getpass.getpass('NASA Earth Data Username:')
nasa_password = getpass.getpass('NASA Earth Data Password:')

In [ ]:
# Run the download command
# This will run in a separate window. Wait for the downlod to complete before proceeding.
# setting auto-file-renaming to false allows incomplete files to be redownloaded
cmd = aria_path + " --auto-file-renaming=false --http-auth-challenge=true --http-user=" + nasa_username + " --http-passwd=" + nasa_password + \
                  " -d " + download_dir + " -i " + url_path
print(cmd)
proc = Popen(cmd, creationflags=CREATE_NEW_CONSOLE)

### Converting Granules to GIS  
The GEDI_subsetter script may be run from the command line to clip granules, get footprint attributes, and save to a GeoJSON files as described on [the script's website](https://git.earthdata.nasa.gov/projects/LPDUR/repos/gedi-subsetter/browse). Here I construct and initiate a command line call to run the subsetter script to clip to the aoi, grab data for a subset of beams and layers, and export to GeoJSON.

**Caveats and Limitations**
1. The "--sds" parameter allows you to export a set of attributes *in addition* to the following set of default attributes. The default attributes are *always included* unless the subsetter script itself modified.  
Default attributes:  
l1bSubset = ['/geolocation/latitude_bin0', '/geolocation/longitude_bin0', '/channel', '/shot_number',
             '/rxwaveform','/rx_sample_count', '/stale_return_flag', '/tx_sample_count', '/txwaveform',
             '/geolocation/degrade', '/geolocation/delta_time', '/geolocation/digital_elevation_model',
              '/geolocation/solar_elevation',  '/geolocation/local_beam_elevation',  '/noise_mean_corrected',
             '/geolocation/elevation_bin0', '/geolocation/elevation_lastbin', '/geolocation/surface_type']
l2aSubset = ['/lat_lowestmode', '/lon_lowestmode', '/channel', '/shot_number', '/degrade_flag', '/delta_time', 
             '/digital_elevation_model', '/elev_lowestmode', '/quality_flag', '/rh', '/sensitivity',  
             '/elevation_bias_flag', '/surface_flag',  '/num_detectedmodes',  '/selected_algorithm',  '/solar_elevation']
l2bSubset = ['/geolocation/lat_lowestmode', '/geolocation/lon_lowestmode', '/channel', '/geolocation/shot_number',
             '/cover', '/cover_z', '/fhd_normal', '/pai', '/pai_z',  '/rhov',  '/rhog',
             '/pavd_z', '/l2a_quality_flag', '/l2b_quality_flag', '/rh100', '/sensitivity',  
             '/stale_return_flag', '/surface_flag', '/geolocation/degrade_flag',  '/geolocation/solar_elevation',
             '/geolocation/delta_time', '/geolocation/digital_elevation_model', '/geolocation/elev_lowestmode']  
             
             
2. A provided ROI shapefile, geojson, or bounding box *MUST* be in WGS84 (epsg:4326). This is assumed by the script and no reprojection is performed of the ROI or the GEDI shots.   
a. If you're primarily working in another coordinate system (especially over a large region) then you may wish to first densify your ROI with more vertices before reprojection to WGS84. Since the GEDI shots themselves are not reprojected for the intersection either, you may accidently exclude points that actually intersect your ROI or accidently include points that don't intersect your ROI. Densifying will reduce this error.
b. As an alternative to densifying, you may wish to follow this alternative workflow to get an exact intersection   
      >1) buffer your ROI then reproject to WGS84  
      >2) run the subsetter tool with the buffered AOI  
      >3) reproject the exported GEDI points to the coordinate system of your original ROI  
      >4) intersect the reprojected points with your original ROI
      
3. A shapefile or GeoJSON ROI must be a single polygon feature. If it contains multiple features only the first feature is used for subsetting. Multipart polygons work, so you can just dissolve the polygons in your ROI file before using the tool.

In [ ]:
# Convert AOI to WGS84 and dissolve to one feature to be useable by GEDI_Subsetter
aoi = gpd.read_file(aoi_path)
aoi_wgs84 = aoi.to_crs(epsg=4326)
aoi_wgs84 = gpd.GeoDataFrame(geometry=[aoi_wgs84.unary_union], crs=4326)
aoi_wgs84.to_file(os.path.join(basedir, 'aoi_wgs84.shp'))

In [ ]:
# setup and run subsetter tool
subsetter_path = r"G:\temp\GEDI_Subsetter.py" # replace with your path to subsetter script

beams = ['BEAM0101','BEAM0110','BEAM1000','BEAM1011']           # full power beams only
sds = ['/selected_l2a_algorithm', '/master_int', '/pgap_theta'] # additional attributes to include

roi_str = os.path.join(basedir, 'aoi_wgs84.shp')
# roi_str = ','.join([str(b) for b in bbox]) # alternatively use bounding box created earlier
beams_str = ','.join(beams)
sds_str = ','.join(sds)


cmd = "python " + subsetter_path + ' --dir ' + download_dir + ' --roi ' +  roi_str + \
      ' --beams ' + beams_str + " --sds " + sds_str
proc = Popen(cmd, creationflags=CREATE_NEW_CONSOLE)

The clipped GeoJSON files will be in a ./output subfolder of the download folder.

## L2B Conversion and Subsetting
The below functions can be used as an alternative to the LPDAAC scripts for subsetting and converting HDF5 files of the GEDI L2B canopy metrics into GeoDataFrames which can then be exported to any GIS output format supported by GeoPandas. This process has a few advantages over the GEDI_Subsetter.py script and a few limitations:

**Advantages**  
1. Much faster overall! For this example area it took 2.5 minutes for the GEDI_Subsetter to run on all beams and default attributes (minus z profiles), but the below L2B conversion and subsequent export only took 6 seconds (2 seconds for subsetting when run in parallel).
2. Can be setup to run in parallell over a folder of files  
3. Automatically dissolves and uses multiple features for subsetting rather than just the first feature
3. Option to subsequently filter laser shots before saving to disk (see quality filtering section below)
4. Option to simply load the data into python memory (not necessary to write the data to a file)  
5. Ability to merge data from multiple HDF files into a single GeoDataframe before saving to disk
6. Option to export to any GeoPandas enabled data format  


**Limitations**
1. Currently only setup to run on L2B files
2. Currently does not work with data stored as 2D arrays in the HDF file. This includes the z profile data (cover_z, pai_z, etc.)
3. As with the GEDI Subsetter tool, this workflow initially filters shots based on the WGS84 projection of the provided ROI, which may exclude or include unwanted shots. You may wish to first buffer or densify your ROI to reduce this problem.

In [ ]:
# Functions for reading GEDI data from HDF files into a GeoDataframe
import h5py, warnings
import geopandas as gpd
import pandas as pd
import numpy as np
from glob import glob
import shapely.speedups
shapely.speedups.enable()

def hdf_dataset_to_column(name, obj):
    """ Append column to the global dataframe "df" if the "obj" is a single column dataset
    # TODO: add ability to get 2-d datasets (e.g. canopy_z)
    """
    if isinstance(obj, h5py._hl.dataset.Dataset):
        basename = name.split('/')[-1]
        if (basename not in bdf) and (obj.shape == (base_len, )):
            bdf[basename] = obj[:]
    return None

def hdf_to_gdf(path, bcols=None, gcols=None, beams=None, aoi=None):
    """ Convert a GEDI hdf file into a geodataframe.
        Note: creating shapely geometries from lat lon is what slows the process down, creating a csv would be a lot faster.
        
        path: str
            path to GEDI hdf file
        bcols: list
            list of beam columns to keep
        gcols: list
            list of geometry columns to keep
        beams: list
            list of beam numbers to keep
        aoi: list or GeoDataframe
            List of geographic (WGS84) bounding box coordinates to filter points in decimal degrees as [minx, miny, maxx, maxy].
            Or, a GeoDataframe of polygons.
        
        Returns: GeoDataFrame
            a geodataframe of data extracted from the GEDI HDF file
            
        # TODO: convert to datatypes specified in hdf
    """
    if not bcols:
        bcols = ['shot_number', 'beam', 'channel', 'l2a_quality_flag', 'l2b_quality_flag', 'sensitivity', 'fhd_normal', 'master_frac', 'master_int', 'cover', 'rh100', 'pai']
    if not gcols:
        gcols = ['elev_lowestmode', 'lat_lowestmode', 'lon_lowestmode', 'degrade_flag', 'solar_elevation']
    if not beams:
        beams = [0, 1, 2, 3, 5, 6, 8, 11]
        
    # setup aoi
    if isinstance(aoi, gpd.GeoDataFrame):
        aoi_wgs84 = aoi.to_crs(epsg=4326)
        bbox = aoi_wgs84.unary_union.bounds
    else:
        bbox = aoi
    
    # read data from hdf
    with h5py.File(path, 'r') as f:
        beam_dfs = {}
        for k, v in f.items():
            if (k.startswith('BEAM')) and (int(k[4:], 2) in beams):
                bdf = pd.DataFrame(index=pd.Series(v['shot_number'][:], name='shot_number'))
                
                if bcols=='all_single':
                    base_len = bdf.shape[0]
                    
                    def hdf_dataset_to_column(name, obj):
                        """ Append column to the global dataframe "df" if the "obj" is a single column dataset
                            This needs to defined locally to update bdf
                        """
                        if isinstance(obj, h5py._hl.dataset.Dataset):
                            basename = name.split('/')[-1]
                            if (basename not in bdf) and (obj.shape == (base_len, )):
                                bdf[basename] = obj[:]
                        return None
                    
                    v.visititems(hdf_dataset_to_column)
                else:
                    for col in bcols:
                        bdf[col] = v[col][:]
                    for col in gcols:
                        bdf[col] = v['geolocation'][col][:]
                
                # simple geometry filtering
                bdf = bdf.dropna(subset=gcols) # drop row if lacking geometry
                if bbox:
                    bdf = bdf[bdf['lat_lowestmode'].between(bbox[1], bbox[3]) & bdf['lon_lowestmode'].between(bbox[0], bbox[2])] # 
                beam_dfs[k] = bdf
    
    # combine and get geometry
    df = pd.concat(beam_dfs.values())
    df = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.lon_lowestmode, df.lat_lowestmode), crs="epsg:4326")
    
    # keep points intersecting the aoi
    if (len(df)>0) and isinstance(aoi, gpd.GeoDataFrame):
        df.sindex
    #         df = df[df.intersects(aoi_wgs84)] # this is much slower than spatial join!
        df = gpd.tools.sjoin(df, aoi_wgs84, how='left', op='intersects')
        df.dropna(subset=['index_right'], inplace=True)
        df.drop('index_right', axis=1, inplace=True)

    # file attributes
    fn = os.path.basename(path)
    df['filename'] = fn
    df['f_year'] = int(fn[9:13])
    df['f_doy'] = int(fn[13:16])
    df['f_hours'] = int(fn[16:18])
    df['f_datetime'] = pd.to_datetime(fn[9:22], format = "%Y%j%H%M%S")
    df['orbit'] = int(fn[24:29])
    df['track'] = int(fn[31:36])
    
    return df

In [ ]:
# Set up input/output
download_dir = os.path.join(basedir, "downloads")
paths = glob(os.path.join(download_dir, '*.h5'))
aoi_path = os.path.join(basedir, 'aoi.shp')
aoi = gpd.read_file(aoi_path)

# Ask for the same set of attributes which are defaults in GEDI_Subsetter.py
bcols = ['shot_number', 'beam', 'channel', 'master_int', 'cover', 'fhd_normal', 'pai', 'rhov', 'rhog',
         'l2a_quality_flag', 'l2b_quality_flag', 'rh100', 'sensitivity', 'stale_return_flag', 'surface_flag']
gcols = ['elev_lowestmode', 'lat_lowestmode', 'lon_lowestmode', 'degrade_flag',  'solar_elevation',
         'delta_time', 'digital_elevation_model']

# full power beams only
beams =  [5, 6, 8, 11] 

In [ ]:
# Convert each hdf in the input folder into a GeoDataFrame through iteration
dfs = {}
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    for i, path in enumerate(paths):
        # Convert the GEDI file to a GeoDataframe
        fn = os.path.basename(path)
        print(fn)
        df = hdf_to_gdf(path, bcols, gcols, beams, aoi)
        dfs[fn] = df

# Merge the shots from all files
df = pd.concat(dfs.values())

In [ ]:
# Or, run on separate files in parallel
from joblib import Parallel, delayed
cores = 4
dfs = Parallel(n_jobs=cores)(delayed(hdf_to_gdf)(path, bcols, gcols, beams, aoi) for path in paths)
df = pd.concat(dfs)

In [ ]:
# plot the aoi and points
import matplotlib.pyplot as plt
fig, ax = plt.subplots(figsize=(10,6))
aoi_wgs84.plot(ax=ax, color='red')
df.plot(ax=ax, color='blue', markersize=2)

**Save to GIS format**  
Note that saving a geopackage with lots of points is very very slow. Saving as a shapefiles isn't recommended since it doesn't preserve the field names or the precision of some datatypes. GeoJSON is also kind of slow and very bulky for large files.  

For a GeoPackage it would be ideal to specify the datatype and precision to use, so the data is less bulky and doesn't get unintentionally get converted from the original type and precision of each attribute in the HDF file, but setting a schema doesn't currently work for all data types. 

If your study area is large and the final dataset will exceed RAM, then it would be wise to export each HDF file upon conversion instead of concatenating them all together in one dataframe for export as done in this example.

In [ ]:
# Save to GeoPackage
outpath = os.path.join(basedir, 'gedi_merged.gpkg')

# # schema for fiona/ogr to write out file (precision not being honored during write)
# properties =  {'shot_number':'int',
#                'beam':'int:8',
#                'channel':'int:3',
#                'l2a_quality_flag':'bool',
#                'l2b_quality_flag':'bool',
#                'sensitivity':'float',
#                'fhd_normal':'float',
#                'master_frac':'float',
#                'master_int':'int:8',
#                'cover':'float:4.4',
#                'rh100':'int:8',
#                'pai':'float',
#                'elev_lowestmode':'float:5.2',
#                'lat_lowestmode':'float:2.7',
#                'lon_lowestmode':'float:3.7',
#                'degrade_flag':'bool',
#                'solar_elevation':'float:3.7',
#                'filename':'str',
#                'f_year':'int:4',
#                'f_doy':'int:3',
#                'f_hours':'int:2',
#                'f_datetime':'str',
#                'orbit':'int:4',
#                'track':'int:4',
#                'shot_str':'str'
#               }
# properties = {k:v for k, v in properties.items() if k in df.columns}
# schema = {'properties':properties, 'geometry':'Point'}

df['shot_str'] = df['shot_number'].astype(str) # shot number is 64-bit int, which technically is not part of the gpkg spec
df.reset_index(drop=True).to_file(outpath, layer="l2b_aoi", driver="GPKG")

**Save to Pickle**  
If the data is only going to be used for analysis in python then a Python pickle is much faster for read/writing and it preserves the precision of all attributes.

In [ ]:
pkl_path = os.path.join(basedir, 'gedi_aoi.pkl')
df.to_pickle(pkl_path)

## L2B Quality filtering  
Each shot includes several attributes for assessing the waveform processing and geolocation accuracy to filter shots that may have erroneous canopy metrics. By running the L2B conversion process above you can apply filters before exporting to save on disk space and export time. 

In [ ]:
# Apply a very conservative filter for high quality shots
sub = df.query(
               'degrade_flag==0 & ' +      # A degrade flag indicates a decrease in geolocation accuracy around the time the shot was taken
    
               'l2b_quality_flag==1 & ' +  # The L2B quality flag combines several other filters; see the docs for details
    
               'solar_elevation<0 & ' +    # Shots at night typically have higher signal:noise ratios
    
               'rh100<5000 & ' +           # A max height greater than 50 m is probably not from a tree
    
#                'slope_pct < 0.3 & ' +      # High slopes can cause the waveform to spread making it harder to detect the ground
    
               'beam >= 5 & ' +            # Power beams can penetrate denser canopies to get the correct ground elevation
    
               'sensitivity >= 0.95'       # Use a higher sensitivity threshold than the L2B quality flag (0.9) if working in areas with very dense canopies
              )

In [ ]:
# plot the aoi and filtered points
fig, ax = plt.subplots(figsize=(10,6))
aoi_wgs84.plot(ax=ax, color='red')
sub.plot(ax=ax, color='blue', markersize=2)

In [ ]:
# Save to GeoPackage
outpath = os.path.join(basedir, 'gedi_merged.gpkg')
sub.reset_index(drop=True).to_file(outpath, layer="l2b_aoi_filtered", driver="GPKG")

The above workflow can be strung together and customized into a single script to automate GEDI preprocessing over large areas or time spans with many files.